In [ ]:
!pip install -q langchain
!pip install -q torch
!pip install -q transformers
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q faiss-cpu
!pip install pypdf

#to run gguf models install below package
# !pip install ctransformers


# To run the GGUF models

In [2]:
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForQuestionAnswering,pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA

In [33]:

#to offload the existing models
import gc
import torch
gc.collect()
torch.cuda.empty_cache()
# try:
torch.cuda.memory_cached()
torch.cuda.reset_peak_memory_stats()
torch.cuda.reset_accumulated_memory_stats()
torch.cuda.reset_max_memory_allocated()
# except:
#     pass

In [4]:
# Embeddings
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

In [5]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/your.pdf")
data = loader.load_and_split()

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=10)
docs = text_splitter.split_documents(data)


In [7]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(docs, embeddings)

In [9]:
from langchain_community.llms import CTransformers
llm = CTransformers(model="TheBloke/zephyr-7B-beta-GGUF", model_type="zephyr-7b-beta.Q4_K_M.gguf")

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm, chain_type="stuff")


In [14]:
query = "Are there any additional requirements for drug treatments?"
docs = db.similarity_search(query)
docs

[Document(page_content='have high quality clinical trial evidence proving it is effective and safe for the treatment  of your \nmedical condition  (full criteria available on request).  \nAre there any additional requirements for drug treatments?   \nIf the treatment  is a drug, the drug must be:  \n• licensed for use by the European Medicines Agency or the Medicines and Healthcare \nproducts Regulatory Agency; and \n• used according to that licence.  \nAre t here any additional requirements for surgical treatments?', metadata={'source': '/content/Businesss Health main handbook 10.23.pdf', 'page': 27}),
 Document(page_content='You will need to read all sections of this handbook to understa nd whether treatment  is eligible  \ntreatment .   \n‘Eligible  treatment ’ is treatment of a disease, illness or injury where that treatment : \n• falls within the benefits of this plan and is not excluded from cover by any term in this handbook; and  \n• is of an acute condition (for details see 3.

In [ ]:

out=chain.run(input_documents=docs, question=query)
print(out)

# To run the Normal models

please note for this we need t4 gpu to run the model

In [2]:
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForQuestionAnswering,pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain import HuggingFacePipeline

In [11]:
# Embeddings
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

In [8]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/Businesss Health main handbook 10.23.pdf")
data = loader.load_and_split()

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
docs = text_splitter.split_documents(data)


In [12]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(docs, embeddings)

In [34]:
llm_tiny_llama = HuggingFacePipeline.from_model_id(
    model_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    task="text-generation",
    device = 0
)

In [35]:
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm_tiny_llama, chain_type="stuff")


In [25]:
query = "What do you mean by conventional treatment?"    ##your query
docs = db.similarity_search(query)
# docs

In [36]:
out=chain.run(input_documents=docs, question=query)
print(out)

 Conventional treatment is the treatment that is established as best medical practice and is practised widely in the UK. It must also be clinically appropriate in terms of necessity, type, frequency, extent, duration, and the facility or location where the treatment is provided. In addition to meeting our definition, it must be approved by NICE (The National Institute for Health and Care Excellence) as a treatment that may be used in routine practice. Otherwise, it must have high quality clinical trial evidence, proving it is effective and safe for the treatment of your medical condition.


In [ ]:
# This ouput is correct, i verified it.